In [1]:
from gluoncv.data import VOCDetection

train_dataset = VOCDetection(root = '../../../data/VOCdevkit',splits=[(2012, 'train')]) # 这是在VOC2012\ImageSets\Main里面找对应名字的txt
val_dataset = VOCDetection(root = '../../../data/VOCdevkit', splits=[(2012, 'val')])

print('Training images:', len(train_dataset))
print('Validation images:', len(val_dataset))

Training images: 5717
Validation images: 5823


In [2]:
from gluoncv import model_zoo
# pretrained_base = True 会加载模型
net = model_zoo.get_model('ssd_300_vgg16_atrous_voc', pretrained_base=False)
print(net)

SSD(
  (features): VGGAtrousExtractor(
    (stages): HybridSequential(
      (0): HybridSequential(
        (0): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Activation(relu)
        (2): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): Activation(relu)
      )
      (1): HybridSequential(
        (0): Conv2D(None -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Activation(relu)
        (2): Conv2D(None -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): Activation(relu)
      )
      (2): HybridSequential(
        (0): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Activation(relu)
        (2): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): Activation(relu)
        (4): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): Activation(relu)
      )
      (3): HybridSeque

In [8]:
# 图像增广对于ssd网络来说很重要，下面就进行增广
from gluoncv.data.transforms import presets
from gluoncv import utils
from mxnet import nd
from gluoncv.data.batchify import Tuple, Stack, Pad
from mxnet.gluon.data import DataLoader
from mxnet import autograd

x = nd.zeros(shape=(1,3,512,512))
net.initialize()

with autograd.train_mode():
    cls_preds, box_preds, anchors = net(x)

width, height = 512, 512  # 假设512*512大小
train_transform = presets.ssd.SSDDefaultTrainTransform(width, height,anchors)
val_transform = presets.ssd.SSDDefaultValTransform(width, height, anchors)

batch_size = 2 # 试运行，弄小点方便看 .训练时调大点。比如128
# you can make it larger(if your CPU has more cores) to accelerate data loading
num_workers = 4 

# behavior of batchify_fn: stack images, and pad labels
batchify_fn = Tuple(Stack(), Stack(), Stack()) # train_transform 有三个参数，这里也需要3个
train_loader = DataLoader(
    train_dataset.transform(train_transform),
    batch_size,
    shuffle=True,
    batchify_fn=batchify_fn,
    last_batch='rollover',
    num_workers=num_workers) # 是否多线程
val_loader = DataLoader(
    val_dataset.transform(val_transform),
    batch_size,
    shuffle=False,
    batchify_fn=batchify_fn,
    last_batch='keep',
    num_workers=num_workers)

d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_vggatrousextractor0_init_scale' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_vggatrousextractor0_conv0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_vggatrousextractor0_conv0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_vggatrousextractor0_conv1_weight' is already initialized, ignoring. Set force_reinit=True t

d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_vggatrousextractor0_normalize0_normalize_scale' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_vggatrousextractor0_extra0_conv0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_vggatrousextractor0_extra0_conv0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_vggatrousextractor0_extra0_conv1_weight' is already initializ

d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_convpredictor3_conv0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_convpredictor3_conv0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_convpredictor5_conv0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
d:\program files\python37\lib\site-packages\mxnet\gluon\parameter.py:813: UserWarning: Parameter 'ssd0_convpredictor5_conv0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v

In [6]:
batch

NameError: name 'batch' is not defined

In [9]:
# from gluoncv.loss import SSDMultiBoxLoss
# mbox_loss = SSDMultiBoxLoss()
# trainer = gluon.Trainer(
#     net.collect_params(), 'sgd',
#     {'learning_rate': 0.001, 'wd': 0.0005, 'momentum': 0.9})

for ib, batch in enumerate(train_loader):
    if ib > 0:
        break
    print('data:', batch[0].shape)
    print('class targets:', batch[1].shape)
    print('box targets:', batch[2].shape)
#     with autograd.record():
#         cls_pred, box_pred, anchors = net(batch[0])
#         sum_loss, cls_loss, box_loss = mbox_loss(
#             cls_pred, box_pred, batch[1], batch[2])
        # some standard gluon training steps:
        # autograd.backward(sum_loss)
        # trainer.step(1)

data: (2, 3, 512, 512)
class targets: (2, 24656)
box targets: (2, 24656, 4)
data: (2, 3, 512, 512)
class targets: (2, 24656)
box targets: (2, 24656, 4)


In [ ]:
train_loader.shape